In [1]:
# importing dependencies
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import faiss
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# creating custom template to guide llm model
custom_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

CUSTOM_QUESTION_PROMPT = PromptTemplate.from_template(custom_template)

In [3]:
# extracting text from pdf
def get_pdf_text(docs):
    text=""
    for pdf in docs:
        pdf_reader=PdfReader(pdf)
        for page in pdf_reader.pages:
            text+=page.extract_text()
    return text

In [4]:
# converting text to chunks
def get_chunks(raw_text):
    text_splitter=CharacterTextSplitter(separator="\n",
                                        chunk_size=100,
                                        chunk_overlap=10,
                                        length_function=len)   
    chunks=text_splitter.split_text(raw_text)
    return chunks

In [5]:
# using all-MiniLm embeddings model and faiss to get vectorstore
def get_vectorstore(chunks):
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                     model_kwargs={'device':'cpu'})
    vectorstore=faiss.FAISS.from_texts(texts=chunks,embedding=embeddings)
    return vectorstore

In [6]:
# generating conversation chain  
def get_conversationchain(vectorstore):
    llm=ChatOpenAI(temperature=0.2)
    memory = ConversationBufferMemory(
        memory_key='chat_history', 
        return_messages=True,
        output_key='answer'
    ) # using conversation buffer memory to hold past information
    conversation_chain = ConversationalRetrievalChain.from_llm(
                                llm=llm,
                                retriever=vectorstore.as_retriever(),
                                condense_question_prompt=CUSTOM_QUESTION_PROMPT,
                                memory=memory)
    return conversation_chain

In [7]:
from pathlib import Path

path = Path("../content/GPT4o.pdf")

text = get_pdf_text([path])
print(text)

What is GPT-4o?
GPT-4o is OpenAI’s latest flagship model that can reason across audio, vision, and text in real time. OpenAI claims it's a step “towards much more natural
human-computer interaction.” 
How is GPT-4o different than other versions of ChatGPT?
GPT-4o differs from other versions of ChatGPT by offering improved performance, including faster response times and better handling of complex queries,
while retaining the comprehensive language capabilities of GPT-4o.
Is GPT-4o free?
Yes, GPT-4o is available to users of the free version of ChatGPT.
Is GPT-4o better than GPT-4o?
GPT-4o is considered an optimized enhancement of GPT-4o, offering better performance in terms of speed and efficiency. However, the core language
capabilities remain consistent with GPT-4o.
Is ChatGPT 4o available?
Yes, GPT-4o is now available to users. 
What does GPT-4o do?
GPT-4o can assess, summarize, and converse with users via text, visuals, and audio. It also answers your queries by combining model know

In [8]:
# Covnerted into Chunks
chunks = get_chunks(text)
print(chunks)

Created a chunk of size 154, which is longer than the specified 100
Created a chunk of size 151, which is longer than the specified 100
Created a chunk of size 145, which is longer than the specified 100
Created a chunk of size 145, which is longer than the specified 100
Created a chunk of size 181, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100


['What is GPT-4o?', "GPT-4o is OpenAI’s latest flagship model that can reason across audio, vision, and text in real time. OpenAI claims it's a step “towards much more natural", 'human-computer interaction.”\xa0\nHow is GPT-4o different than other versions of ChatGPT?', 'GPT-4o differs from other versions of ChatGPT by offering improved performance, including faster response times and better handling of complex queries,', 'while retaining the comprehensive language capabilities of GPT-4o.\nIs GPT-4o free?', 'Yes, GPT-4o is available to users of the free version of ChatGPT.\nIs GPT-4o better than GPT-4o?', 'GPT-4o is considered an optimized enhancement of GPT-4o, offering better performance in terms of speed and efficiency. However, the core language', 'capabilities remain consistent with GPT-4o.\nIs ChatGPT 4o available?', 'Yes, GPT-4o is now available to users.\xa0\nWhat does GPT-4o do?', 'GPT-4o can assess, summarize, and converse with users via text, visuals, and audio. It also answ

In [9]:
vectorstore = get_vectorstore(chunks)
print(vectorstore)

In [10]:
chain = get_conversationchain(vectorstore)
print(chain)

memory=ConversationBufferMemory(output_key='answer', return_messages=True, memory_key='chat_history') combine_docs_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000024914D0AD20>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000249124D8A40>, root_client=<openai.OpenAI object at 0x0000024965D3BFE0>, root_async_client=<openai.AsyncOpenAI object at 0x0000024914D0AD80>, temperature=0.2, openai_api_key=SecretStr('********

In [11]:
response = chain({
    "question": "Key features of GPT4o?"
})

In [12]:
response

{'question': 'Key features of GPT4o?',
 'chat_history': [HumanMessage(content='Key features of GPT4o?'),
  AIMessage(content='The key features of GPT-4o include improved performance in terms of speed and efficiency compared to its predecessor, GPT-4. It maintains the robust language capabilities of the previous version while offering enhancements in overall functionality.')],
 'answer': 'The key features of GPT-4o include improved performance in terms of speed and efficiency compared to its predecessor, GPT-4. It maintains the robust language capabilities of the previous version while offering enhancements in overall functionality.'}

In [16]:
print(response['question'])
print("#"*40)
print(response['chat_history'])
print("#"*40)
print(response['answer'])
print("#"*40)

Key features of GPT4o?
########################################
[HumanMessage(content='Key features of GPT4o?'), AIMessage(content='The key features of GPT-4o include improved performance in terms of speed and efficiency compared to its predecessor, GPT-4. It maintains the robust language capabilities of the previous version while offering enhancements in overall functionality.')]
########################################
The key features of GPT-4o include improved performance in terms of speed and efficiency compared to its predecessor, GPT-4. It maintains the robust language capabilities of the previous version while offering enhancements in overall functionality.
########################################


In [17]:
response1 = chain({
    "question": "What is new about in GPT4o?"
})

In [18]:
print(response1['question'])
print("#"*40)
print(response1['chat_history'])
print("#"*40)
print(response1['answer'])
print("#"*40)

What is new about in GPT4o?
########################################
[HumanMessage(content='Key features of GPT4o?'), AIMessage(content='The key features of GPT-4o include improved performance in terms of speed and efficiency compared to its predecessor, GPT-4. It maintains the robust language capabilities of the previous version while offering enhancements in overall functionality.'), HumanMessage(content='What is new about in GPT4o?'), AIMessage(content="GPT-4o is a new, optimized version of OpenAI's GPT-4, designed to enhance performance and efficiency while maintaining its predecessor's robust language capabilities.")]
########################################
GPT-4o is a new, optimized version of OpenAI's GPT-4, designed to enhance performance and efficiency while maintaining its predecessor's robust language capabilities.
########################################
